In [3]:
# Import packages
import os
import random
from PIL import Image
from torch import nn

from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
image_path = RAW_DATA_DIR
train_dir = image_path / 'train'
test_dir = image_path / 'test'

In [1]:
import json

config_path = 'params.json'
with open(config_path, 'r') as config_file:
    config = json.load(config_file)
print(json.dumps(config, indent=2))

{
  "model_params": {
    "learning_rate": 0.001,
    "batch_size": 64,
    "num_epochs": 25,
    "dropout_rate": 0.3,
    "optimizer": "adam",
    "loss_function": "categorical_crossentropy",
    "metrics": [
      "accuracy"
    ]
  }
}


In [4]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']

In [5]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
class HHP01(nn.Module):
    def __init__(self):
        super(HHP01, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten()
            nn.Linear()
            nn.Softmax(dim=1)
        )